In [161]:
import rioxarray, numpy as np

rds = rioxarray.open_rasterio("uzemicko.tif")
rds = rds.to_dataframe(name="slope")
print(rds)

                                 spatial_ref     slope
band y             x                                  
1    -1.228974e+06 -257921.4999            0  1.382104
                   -257920.4999            0  0.381123
                   -257919.4999            0  0.500941
                   -257918.4999            0  0.777611
                   -257917.4999            0  1.050895
...                                      ...       ...
     -1.231501e+06 -254073.4999            0  3.321621
                   -254072.4999            0  3.734349
                   -254071.4999            0  3.944973
                   -254070.4999            0  3.939653
                   -254069.4999            0  3.893767

[9740384 rows x 2 columns]


In [162]:
df = rds[:10000]

In [163]:
print(df)

                                 spatial_ref     slope
band y             x                                  
1    -1.228974e+06 -257921.4999            0  1.382104
                   -257920.4999            0  0.381123
                   -257919.4999            0  0.500941
                   -257918.4999            0  0.777611
                   -257917.4999            0  1.050895
...                                      ...       ...
     -1.228976e+06 -255632.4999            0  6.401651
                   -255631.4999            0  7.513536
                   -255630.4999            0  6.971714
                   -255629.4999            0  5.040220
                   -255628.4999            0  2.467881

[10000 rows x 2 columns]


In [164]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import shapefile

polygons = []
sf = shapefile.Reader("slope_deformations_reference_polygons/reference_data_existing_landslides_polygons.shp")
shapes = sf.shapes()
rectangles = []

for shape in shapes:
    polygons.append(Polygon(shape.points))
    rectangles.append(shape.bbox)

In [165]:
def is_in_optimised(x, y):
    for i in range(len(polygons)):
        rectangle = rectangles[i]
        low_x, low_y, high_x, high_y = rectangle[0], rectangle[1], rectangle[2], rectangle[3]
        if x >= low_x and x <= high_x and y >= low_y and y <= high_y:
            polygon = polygons[i]
            if polygon.contains(Point(x, y)):
                return True

    return False

In [166]:
is_landslide = set()
for i in range(len(polygons)):
    rectangle = rectangles[i]
    low_x, low_y, high_x, high_y = rectangle[0], rectangle[1], rectangle[2], rectangle[3]
    small_df = df.query('y >= @low_y and y <= @high_y and x >= @low_x and x <= @high_x')
    if(len(small_df) > 0):
        for _, y, x in small_df.index:
            if polygons[i].contains(Point(x, y)):
                is_landslide.add((x, y))

In [167]:
bools = np.array([True if (x, y) in is_landslide else False for _, y, x in df.index])
df.insert(loc = 0, column='is_in', value=bools)
print(df)

                                 is_in  spatial_ref     slope
band y             x                                         
1    -1.228974e+06 -257921.4999  False            0  1.382104
                   -257920.4999  False            0  0.381123
                   -257919.4999  False            0  0.500941
                   -257918.4999  False            0  0.777611
                   -257917.4999  False            0  1.050895
...                                ...          ...       ...
     -1.228976e+06 -255632.4999   True            0  6.401651
                   -255631.4999   True            0  7.513536
                   -255630.4999   True            0  6.971714
                   -255629.4999   True            0  5.040220
                   -255628.4999   True            0  2.467881

[10000 rows x 3 columns]
